In [2]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import pandas as pd
import torch

C:\Users\Admin\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# task='sentiment'
# MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

# tokenizer = AutoTokenizer.from_pretrained(MODEL)
# #tokenizer = AutoTokenizer.from_pretrained("./tokenizer_roberta/token_izer/")

# #tokenizer.save_pretrained("tokenizer_roberta/token_izer")

from transformers import AutoTokenizer
import urllib.request

task = "sentiment"
#MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

# Correct way to load tokenizer
#tokenizer = AutoTokenizer.from_pretrained(MODEL)
#tokenizer = AutoTokenizer.from_pretrained("./tokenizer_roberta/sentiment_tokenizer/")
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")


#tokenizer = AutoTokenizer.from_pretrained("C:/Users/Admin/Downloads/SIH-2023-main/SIH-2023-main/server/tokenizer_roberta/sentiment_tokenizer/")




# If you want to save and load locally:
# tokenizer.save_pretrained("tokenizer_roberta/token_izer")
# tokenizer = AutoTokenizer.from_pretrained("tokenizer_roberta/token_izer")  # Use this after saving




# download label mapping
labels=[]

mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
#labels = [row[1] for row in csvreader if len(row) > 1]
try:
    with urllib.request.urlopen(mapping_link) as f:
        html = f.read().decode('utf-8').split("\n")
        csvreader = csv.reader(html, delimiter="\t")
        labels = [row[1] for row in csvreader if len(row) > 1]
except Exception as e:
    print(f"Error downloading label mapping: {e}")
    labels = ["negative", "neutral", "positive"]  # Fallback


model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

C:\Users\Admin\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--cardiffnlp--twitter-roberta-base-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


OSError: Incorrect path_or_model_id: './tokenizer_roberta/token_izer/'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("./tokenizer_roberta/sentiment_tokenizer/")
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")


In [ ]:
#model = AutoModelForSequenceClassification.from_pretrained("./cardiffnlp/twitter-roberta-base-sentiment/")
model = AutoModelForSequenceClassification.from_pretrained(MODEL)


In [ ]:
def sentiment(row):
    text = row[:1500]
    encoded_input = tokenizer(text, return_tensors='pt')
    with torch.no_grad():  # Disable gradient calculation during inference
        output = model(**encoded_input)
    # scores = output[0][0].cpu().numpy()  # Move scores to CPU for further processing
    # scores = softmax(scores)
    scores = output.logits[0].cpu().numpy()
    scores = softmax(scores)

    

    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    max_score = 0
    for i in range(scores.shape[0]):
        l = labels[ranking[i]]
        s = scores[ranking[i]]
        if s > max_score:
            final = l
            max_score = s
    return final

### News18

In [ ]:
# df = pd.read_excel(r"C:\Users\Dhruv\Downloads\sih\IndiaToday_Preprocessed.xlsx")
df = pd.read_excel("C:\\Users\\Dhruv\\Downloads\\sih\\IndiaToday_Preprocessed.xlsx")


In [25]:
df = df.iloc[:100, :]

In [ ]:
#df["Sentiment"] = df.Body.apply(lambda x: sentiment(str(x)))
df["Sentiment"] = df.Body.fillna("").apply(lambda x: sentiment(str(x)))


In [ ]:
df.head()

,Heading,Body,Category,URL,Sentiment
0,'Free speech should not be hate speech': Madra...,madras high court issue significant remark ami...,law,https://www.indiatoday.in/law/high-courts/stor...,neutral
1,"'Comments taken out of context', says US cop w...",seattle police officer guild friday come defen...,world,https://www.indiatoday.in/world/story/indian-s...,neutral
2,First meeting of 'One Nation One Election' com...,first official meeting one nation one election...,india,https://www.indiatoday.in/india/story/one-nati...,neutral
3,"US airlines’ flight depressurizes mid-air, plu...",united airlines jet head rome turn around less...,world,https://www.indiatoday.in/world/story/us-fligh...,neutral
4,3 terrorists killed as security forces foil in...,three terrorist kill infiltration bid foil sec...,india,https://www.indiatoday.in/india/story/one-terr...,negative


In [ ]:
#df.head()

,Heading,Body,Category,URL,Sentiment
0,'Free speech should not be hate speech': Madra...,madras high court issue significant remark ami...,law,https://www.indiatoday.in/law/high-courts/stor...,neutral
1,"'Comments taken out of context', says US cop w...",seattle police officer guild friday come defen...,world,https://www.indiatoday.in/world/story/indian-s...,neutral
2,First meeting of 'One Nation One Election' com...,first official meeting one nation one election...,india,https://www.indiatoday.in/india/story/one-nati...,neutral
3,"US airlines’ flight depressurizes mid-air, plu...",united airlines jet head rome turn around less...,world,https://www.indiatoday.in/world/story/us-fligh...,neutral
4,3 terrorists killed as security forces foil in...,three terrorist kill infiltration bid foil sec...,india,https://www.indiatoday.in/india/story/one-terr...,negative


In [80]:
file_name = "News18_with_sentiment.xlsx"
df.to_excel(file_name, index=False)

### IndiaToday 

In [5]:
# df2 = pd.read_excel(r"C:\Users\Dhruv\Downloads\sih\IndiaToday_Preprocessed.xlsx")
df2 = pd.read_excel("C:\\Users\\Dhruv\\Downloads\\IndiaToday_Preprocessed.xlsx")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Dhruv\\Downloads\\IndiaToday_Preprocessed.xlsx'

In [92]:
df2["Sentiment"] = df2.Body.apply(lambda x: sentiment(str(x)))

In [93]:
file_name = "IndiaToday_with_sentiment.xlsx"
df2.to_excel(file_name, index=False)

### IndiaTV

In [88]:
df3 = pd.read_excel(r"C:\Users\Dhruv\Downloads\sih\IndiaTV_Preprocessed.xlsx")

In [90]:
df3["Sentiment"] = df3.Body.apply(lambda x: sentiment(x))

In [91]:
file_name = "IndiaTV_with_sentiment.xlsx"
df3.to_excel(file_name, index=False)